# Final project for the Capstone course

### Problem:
The city of Regensburg is a small, academic, medieval, and historic city, due to this the not only the tourism around it but the flow of students, in times without the pandemic, are constant. The city is structured into six districts, some more oriented for academic purposes and some for tourism. 
The problem to address in this notebook is which district is the best to open a Cofé shoop in the city, taking into account the distribution of the current Cofé shops around the city.

# Metodology:
The methodology to solve the problem will be:
- Determine the six districts using its Postal Code, latitude, and longitude using the Deutsche Post website. 
- Determine the number of Cofé shops in each district and its number of tips using the Foursquare API. this to determine the flow of people for each district.
- Determine the best district, after the information above has been.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
#df = pd.read_csv('/home/hector/Dropbox/courses/data-sci/08-App-ds-caps/covid-19-germany-landkreise.csv', sep=';')
#df

In [6]:
#df[df["Name"] == 'Regensburg']

### Creating the Dataframe using the csv file.


In [2]:
df_r = pd.read_csv('/home/hector/Dropbox/courses/data-sci/08-App-ds-caps/regens.csv')
df_r

,PostalCode,City,District,Latitude,Longitude
0,93051,Regensburg,Großprüfening-Dechbetten-Königswiesen,49.007839,12.077042
1,93047,Regensburg,Innenstadt,49.016491,12.100904
2,93053,Regensburg,Kasernenviertel,49.012688,12.105520
3,93055,Regensburg,Ostenviertel,49.016174,12.107655
4,93049,Regensburg,Westenviertel,49.012512,12.066780
5,93055,Regensburg,Brandlberg-Keilberg,49.040813,12.152743
6,93053,Regensburg,Galgenberg,48.993454,12.110230


In [10]:
df_r.dtypes

PostalCode      int64
City           object
District       object
Latitude      float64
Longitude     float64
dtype: object

In [33]:
address = 'Regensburg, DE'

geolocator = Nominatim(user_agent="reg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Regensburg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Regensburg are 49.0195333, 12.0974869.


In [39]:
#create map of New York using latitude and longitude values
map_regens = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_r['Latitude'], df_r['Longitude'], df_r['District']):
    #print(toronto_data.iloc[2])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_regens)  
    
map_regens

In [41]:
CLIENT_ID = '5YMOYTKQ5D5UA5ZLLUZEQL4GTTVOBZOJTDUPJJCQ1ISVEGJL' # your Foursquare ID
CLIENT_SECRET = 'N0231QPA5OQT3V5KCP1S1K4NDIORUFLCSZXAYJOAWE51PVZX' # your Foursquare Secret
ACCESS_TOKEN = 'DIMOB4IDVP1HC1NZOFISJN5FS1IS4AKDVSEKRRWB3FMDWTTP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5YMOYTKQ5D5UA5ZLLUZEQL4GTTVOBZOJTDUPJJCQ1ISVEGJL
CLIENT_SECRET:N0231QPA5OQT3V5KCP1S1K4NDIORUFLCSZXAYJOAWE51PVZX


In [77]:
#address = 'Kasernenviertel, Regensburg, DE'

#geolocator = Nominatim(user_agent="foursquare_agent")
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
#print(latitude, longitude)

In [164]:
search_query = 'Café'
radius = 1000
print(search_query + ' .... OK!')

Café .... OK!


# Café: Innerstadt

In [168]:
latitude_inn = 49.016491
longitude_inn = 12.100904
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_inn, longitude_inn,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

print(latitude_inn,longitude_inn)
results_inn = requests.get(url).json()

# assign relevant part of JSON to venues
venues_inn = results_inn['response']['venues']

# tranform venues into a dataframe
df_inn = json_normalize(venues_inn)
df_inn.head()

49.016491 12.100904


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4db813be93a04a7c3078118b,Café Lola,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739036,False,Pfarrergasse 6,49.017346,12.095911,"[{'label': 'display', 'lat': 49.01734641290232...",376,93047,DE,Regensburg,Bayern,Deutschland,"[Pfarrergasse 6, 93047 Regensburg]",NaN,NaN
1,5149a91b45b0a6d087bba29f,Cafe Verschmitzt,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739036,False,Hemauer Strasse 6,49.013245,12.102492,"[{'label': 'display', 'lat': 49.01324496519724...",379,93047,DE,Regensburg,Bayern,Deutschland,"[Hemauer Strasse 6, 93047 Regensburg]",57123933,NaN
2,4b6c7b44f964a5209d3d2ce3,Café Scholz,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739036,False,D.-Martin-Luther-Str. 2,49.016570,12.101370,"[{'label': 'display', 'lat': 49.01656975144709...",35,93047,DE,Regensburg,Bayern,Deutschland,"[D.-Martin-Luther-Str. 2 (Dachauplatz), 93047 ...",NaN,Dachauplatz
3,54140ef8498e79de147ff0b6,190 GRAD CAFÉ BAR RÖSTEREI Rehorik,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739036,False,Brixener Hof,49.016959,12.099510,"[{'label': 'display', 'lat': 49.016959, 'lng':...",114,93047,DE,Regensburg,Bayern,Deutschland,"[Brixener Hof, 93047 Regensburg]",NaN,NaN
4,4cd6d4ed122ba1435efb34a1,Cafe Fürstenhof,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739036,False,NaN,49.016450,12.099384,"[{'label': 'display', 'lat': 49.01645, 'lng': ...",111,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,NaN


In [169]:
df_inn.shape

(30, 18)

In [170]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_inn.columns if col.startswith('location.')] + ['id']
df_inn_filtered = df_inn.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_inn_filtered['categories'] = df_inn_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_inn_filtered.columns = [column.split('.')[-1] for column in df_inn_filtered.columns]

df_inn_filtered.head(5)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Café Lola,Café,Pfarrergasse 6,49.017346,12.095911,"[{'label': 'display', 'lat': 49.01734641290232...",376,93047,DE,Regensburg,Bayern,Deutschland,"[Pfarrergasse 6, 93047 Regensburg]",NaN,4db813be93a04a7c3078118b
1,Cafe Verschmitzt,Café,Hemauer Strasse 6,49.013245,12.102492,"[{'label': 'display', 'lat': 49.01324496519724...",379,93047,DE,Regensburg,Bayern,Deutschland,"[Hemauer Strasse 6, 93047 Regensburg]",NaN,5149a91b45b0a6d087bba29f
2,Café Scholz,Café,D.-Martin-Luther-Str. 2,49.016570,12.101370,"[{'label': 'display', 'lat': 49.01656975144709...",35,93047,DE,Regensburg,Bayern,Deutschland,"[D.-Martin-Luther-Str. 2 (Dachauplatz), 93047 ...",Dachauplatz,4b6c7b44f964a5209d3d2ce3
3,190 GRAD CAFÉ BAR RÖSTEREI Rehorik,Café,Brixener Hof,49.016959,12.099510,"[{'label': 'display', 'lat': 49.016959, 'lng':...",114,93047,DE,Regensburg,Bayern,Deutschland,"[Brixener Hof, 93047 Regensburg]",NaN,54140ef8498e79de147ff0b6
4,Cafe Fürstenhof,Café,NaN,49.016450,12.099384,"[{'label': 'display', 'lat': 49.01645, 'lng': ...",111,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,4cd6d4ed122ba1435efb34a1


# café: Großprüfening-Dechbetten-Königswiesen

In [171]:
#latitude_grode = 49.007839
#longitude_grode = 12.077042
latitude_grode =df_r.iloc[0,3]
longitude_grode =df_r.iloc[0,4]

url_grode = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_grode, longitude_grode,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url_grode

'https://api.foursquare.com/v2/venues/search?client_id=5YMOYTKQ5D5UA5ZLLUZEQL4GTTVOBZOJTDUPJJCQ1ISVEGJL&client_secret=N0231QPA5OQT3V5KCP1S1K4NDIORUFLCSZXAYJOAWE51PVZX&ll=49.007839000000004,12.077042&oauth_token=DIMOB4IDVP1HC1NZOFISJN5FS1IS4AKDVSEKRRWB3FMDWTTP&v=20180604&query=Café&radius=1000&limit=30'

In [173]:
results_grode = requests.get(url_grode).json()
results_grode

# assign relevant part of JSON to venues
venues_grode = results_grode['response']['venues']

# tranform venues into a dataframe
df_grode = json_normalize(venues_grode)
df_grode.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,5c58a223cb3fd2002cfb52ce,Josef Schwarz Konditorei-Café Mandl,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739066,False,Hofgartenweg 2,49.006847,12.089587,"[{'label': 'display', 'lat': 49.006847, 'lng':...",922,93051,DE,Regensburg,Bayern,Deutschland,"[Hofgartenweg 2, 93051 Regensburg]"
1,4fc0eac6e4b01d437dda10b2,Cafe Rosarium,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739066,False,NaN,49.015310,12.084037,"[{'label': 'display', 'lat': 49.01531040957402...",975,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg]
2,4de3ef70b0fb267148a8f960,Postfiliale Bäckerei-Cafe Westwärts,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1608739066,False,Prüfeninger Straße 62,49.017037,12.075528,"[{'label': 'display', 'lat': 49.01703678514465...",1029,93049,DE,Regensburg,Bayern,Deutschland,"[Prüfeninger Straße 62, 93049 Regensburg]"
3,512732b1e4b077f708c7e12a,Cafe La Vie,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1608739066,False,Ziegetsdorferstr. 115,49.010158,12.058706,"[{'label': 'display', 'lat': 49.010158, 'lng':...",1363,NaN,DE,Regensburg,Bayern,Deutschland,"[Ziegetsdorferstr. 115, Regensburg]"
4,4fe5a699e4b0daa8b5a80906,Gäuboden Bäcker,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1608739066,False,NaN,49.010786,12.058554,"[{'label': 'display', 'lat': 49.01078575003176...",1389,NaN,DE,NaN,NaN,Deutschland,NaN


In [175]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_grode = ['name', 'categories'] + [col for col in df_grode.columns if col.startswith('location.')] + ['id']
df_grode_filtered = df_grode.loc[:, filtered_columns_grode]

# filter the category for each row
df_grode_filtered['categories'] = df_grode_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_grode_filtered.columns = [column.split('.')[-1] for column in df_grode_filtered.columns]

df_grode_filtered.head(5)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Josef Schwarz Konditorei-Café Mandl,Café,Hofgartenweg 2,49.006847,12.089587,"[{'label': 'display', 'lat': 49.006847, 'lng':...",922,93051,DE,Regensburg,Bayern,Deutschland,"[Hofgartenweg 2, 93051 Regensburg]",5c58a223cb3fd2002cfb52ce
1,Cafe Rosarium,Café,NaN,49.015310,12.084037,"[{'label': 'display', 'lat': 49.01531040957402...",975,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],4fc0eac6e4b01d437dda10b2
2,Postfiliale Bäckerei-Cafe Westwärts,Bakery,Prüfeninger Straße 62,49.017037,12.075528,"[{'label': 'display', 'lat': 49.01703678514465...",1029,93049,DE,Regensburg,Bayern,Deutschland,"[Prüfeninger Straße 62, 93049 Regensburg]",4de3ef70b0fb267148a8f960
3,Cafe La Vie,Café,Ziegetsdorferstr. 115,49.010158,12.058706,"[{'label': 'display', 'lat': 49.010158, 'lng':...",1363,NaN,DE,Regensburg,Bayern,Deutschland,"[Ziegetsdorferstr. 115, Regensburg]",512732b1e4b077f708c7e12a
4,Gäuboden Bäcker,Bakery,NaN,49.010786,12.058554,"[{'label': 'display', 'lat': 49.01078575003176...",1389,NaN,DE,NaN,NaN,Deutschland,NaN,4fe5a699e4b0daa8b5a80906


In [176]:
df_grode_filtered.shape

(5, 14)

# Café: Kasernenviertel 

In [158]:
#latitude_kas = 49.012688
#longitude_kas = 12.105520
latitude_kas = df_r.iloc[2,3]
longitude_kas = df_r.iloc[2,4]


url_kas = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_kas, longitude_kas,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results_kas = requests.get(url_kas).json()

# assign relevant part of JSON to venues
venues_kas = results_kas['response']['venues']

# tranform venues into a dataframe
df_kas = json_normalize(venues_kas)
df_kas.shape

(9, 18)

In [159]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_kas = ['name', 'categories'] + [col for col in df_kas.columns if col.startswith('location.')] + ['id']
df_kas_filtered = df_kas.loc[:, filtered_columns_kas]

# filter the category for each row
df_kas_filtered['categories'] = df_kas_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_kas_filtered.columns = [column.split('.')[-1] for column in df_kas_filtered.columns]

df_kas_filtered.head(5)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Cafe Verschmitzt,Café,Hemauer Strasse 6,49.013245,12.102492,"[{'label': 'display', 'lat': 49.01324496519724...",229,93047,DE,Regensburg,Bayern,Deutschland,"[Hemauer Strasse 6, 93047 Regensburg]",NaN,5149a91b45b0a6d087bba29f
1,Cafe Samer,Hookah Bar,NaN,49.014793,12.105973,"[{'label': 'display', 'lat': 49.01479271829082...",236,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,52a0e09a498e8e44e72394f6
2,Cafè Züchtig,Café,NaN,49.015835,12.106932,"[{'label': 'display', 'lat': 49.01583533449219...",365,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,4b8fd50ff964a520826433e3
3,Beyoğlu Internet Cafe,Nightclub,R,49.013275,12.101644,"[{'label': 'display', 'lat': 49.01327514648437...",290,NaN,DE,NaN,NaN,Deutschland,[R (T)],T,55e0bafb498e6ef8d8f9b0e3
4,Cafe Pernsteiner,None,NaN,49.016818,12.107558,"[{'label': 'display', 'lat': 49.01681838817746...",483,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,4b96a398f964a5209eda34e3


In [160]:
df_kas_filtered.shape

(9, 15)

# Café: Ostenviertel 

In [177]:
latitude_ost = df_r.iloc[3,3]
longitude_ost = df_r.iloc[3,4]


url_ost = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ost, longitude_ost,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results_ost = requests.get(url_ost).json()

# assign relevant part of JSON to venues
venues_ost = results_ost['response']['venues']

# tranform venues into a dataframe
df_ost = json_normalize(venues_ost)
df_ost.shape

(30, 18)

In [178]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_ost = ['name', 'categories'] + [col for col in df_ost.columns if col.startswith('location.')] + ['id']
df_ost_filtered = df_ost.loc[:, filtered_columns_ost]

# filter the category for each row
df_ost_filtered['categories'] = df_ost_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_ost_filtered.columns = [column.split('.')[-1] for column in df_ost_filtered.columns]

df_ost_filtered.head(5)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Cafe Verschmitzt,Café,Hemauer Strasse 6,49.013245,12.102492,"[{'label': 'display', 'lat': 49.01324496519724...",498,93047,DE,Regensburg,Bayern,Deutschland,"[Hemauer Strasse 6, 93047 Regensburg]",NaN,5149a91b45b0a6d087bba29f
1,Café Lola,Café,Pfarrergasse 6,49.017346,12.095911,"[{'label': 'display', 'lat': 49.01734641290232...",867,93047,DE,Regensburg,Bayern,Deutschland,"[Pfarrergasse 6, 93047 Regensburg]",NaN,4db813be93a04a7c3078118b
2,Cafè Züchtig,Café,NaN,49.015835,12.106932,"[{'label': 'display', 'lat': 49.01583533449219...",64,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,4b8fd50ff964a520826433e3
3,Café Lederer,Café,Weichser Weg 5,49.025634,12.115424,"[{'label': 'display', 'lat': 49.02563445620731...",1196,93059,DE,Regensburg,Bayern,Deutschland,"[Weichser Weg 5, 93059 Regensburg]",NaN,4ce79f14fe90a35d11843d0e
4,Cafe Pernsteiner,None,NaN,49.016818,12.107558,"[{'label': 'display', 'lat': 49.01681838817746...",72,NaN,DE,Regensburg,Bayern,Deutschland,[Regensburg],NaN,4b96a398f964a5209eda34e3


In [163]:
df_ost_filtered.shape

(9, 15)

# Café: Westenviertel

In [179]:
latitude_wes = df_r.iloc[4,3]
longitude_wes = df_r.iloc[4,4]


url_wes = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_wes, longitude_wes,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results_wes = requests.get(url_wes).json()

# assign relevant part of JSON to venues
venues_wes = results_wes['response']['venues']

# tranform venues into a dataframe
df_wes = json_normalize(venues_wes)
print("Original shape",df_wes.shape)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_wes = ['name', 'categories'] + [col for col in df_wes.columns if col.startswith('location.')] + ['id']
df_wes_filtered = df_wes.loc[:, filtered_columns_wes]

# filter the category for each row
df_wes_filtered['categories'] = df_wes_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_wes_filtered.columns = [column.split('.')[-1] for column in df_wes_filtered.columns]

print("Shape post-filter", df_wes_filtered.shape)
df_wes_filtered.head(5)

Original shape (6, 17)
Shape post-filter (6, 15)


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,crossStreet,id
0,Cafe La Vie,Café,Ziegetsdorferstr. 115,49.010158,12.058706,"[{'label': 'display', 'lat': 49.010158, 'lng':...",645,DE,Regensburg,Bayern,Deutschland,"[Ziegetsdorferstr. 115, Regensburg]",NaN,NaN,512732b1e4b077f708c7e12a
1,Cafe Königsgarten,Gastropub,Udetstrasse,49.018852,12.057432,"[{'label': 'display', 'lat': 49.01885199431989...",981,DE,Regensburg,Bayern,Deutschland,"[Udetstrasse, 93049 Regensburg]",93049,NaN,519f4144498e0e64c6fb64c7
2,Postfiliale Bäckerei-Cafe Westwärts,Bakery,Prüfeninger Straße 62,49.017037,12.075528,"[{'label': 'display', 'lat': 49.01703678514465...",813,DE,Regensburg,Bayern,Deutschland,"[Prüfeninger Straße 62, 93049 Regensburg]",93049,NaN,4de3ef70b0fb267148a8f960
3,Cafeteria HS.R Prüfening,College Cafeteria,NaN,49.019260,12.059527,"[{'label': 'display', 'lat': 49.01926032256866...",919,DE,NaN,NaN,Deutschland,NaN,NaN,NaN,507e85c2e4b0225db10a02fc
4,Gäuboden Bäcker,Bakery,NaN,49.010786,12.058554,"[{'label': 'display', 'lat': 49.01078575003176...",630,DE,NaN,NaN,Deutschland,NaN,NaN,NaN,4fe5a699e4b0daa8b5a80906


# Café: Brandlberg-Keilberg

In [180]:
latitude_bra = df_r.iloc[5,3]
longitude_bra = df_r.iloc[5,4]


url_bra = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_bra, longitude_bra,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results_bra = requests.get(url_bra).json()

# assign relevant part of JSON to venues
venues_bra = results_bra['response']['venues']

# tranform venues into a dataframe
df_bra = json_normalize(venues_bra)
print("Original shape",df_bra.shape)

# keep only columns that include venue name, and anything that is associated with location
#filtered_columns_bra = ['name', 'categories'] + [col for col in df_bra.columns if col.startswith('location.')] + ['id']
#df_bra_filtered = df_bra.loc[:, filtered_columns_bra]

# filter the category for each row
#df_bra_filtered['categories'] = df_bra_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
#df_bra_filtered.columns = [column.split('.')[-1] for column in df_bra_filtered.columns]

#print("Shape post-filter", df_bra_filtered.shape)
#df_bra_filtered.head(5)

Original shape (0, 0)


# Café: Galgenberg

In [181]:
latitude_gal = df_r.iloc[6,3]
longitude_gal = df_r.iloc[6,4]


url_gal = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_gal, longitude_gal,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results_gal = requests.get(url_gal).json()

# assign relevant part of JSON to venues
venues_gal = results_gal['response']['venues']

# tranform venues into a dataframe
df_gal = json_normalize(venues_gal)
print("Original shape",df_gal.shape)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_gal = ['name', 'categories'] + [col for col in df_gal.columns if col.startswith('location.')] + ['id']
df_gal_filtered = df_gal.loc[:, filtered_columns_gal]

# filter the category for each row
df_gal_filtered['categories'] = df_gal_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_gal_filtered.columns = [column.split('.')[-1] for column in df_gal_filtered.columns]

print("Shape post-filter", df_gal_filtered.shape)
df_gal_filtered.head(5)

Original shape (1, 14)
Shape post-filter (1, 12)


,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,id
0,Chemie Cafeteria,College Cafeteria,48.996034,12.095468,"[{'label': 'display', 'lat': 48.99603375709827...",1115,DE,Regensburg,Bayern,Deutschland,[Regensburg],50feae4ae4b038294a17bb94
